In [ ]:
import quanguru as qg
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
numOfQubits = 5
freequency = 0
freeOperator = qg.Jz
couplingStrength = 1

In [ ]:
nQubExchange = numOfQubits * qg.Qubit(frequency=freequency, operator=freeOperator)
nQubHeisenberg = numOfQubits * qg.Qubit(frequency=freequency, operator=freeOperator)

In [ ]:
exchangeQubs = list(nQubExchange.subSys.values())
for ind in range(numOfQubits-1):
    es = [exchangeQubs[ind], exchangeQubs[ind+1]]
    nQubExchange.createTerm(operator=[qg.sigmax, qg.sigmax], frequency=0, qSystem=es)
    nQubExchange.createTerm(operator=[qg.sigmay, qg.sigmay], frequency=0, qSystem=es)

In [ ]:
HeisenbergQubs = list(nQubHeisenberg.subSys.values())
for ind in range(numOfQubits-1):
    hs = [HeisenbergQubs[ind], HeisenbergQubs[ind+1]]
    nQubHeisenberg.createTerm(operator=[qg.sigmax, qg.sigmax], frequency=couplingStrength,qSystem=hs)
    nQubHeisenberg.createTerm(operator=[qg.sigmay, qg.sigmay], frequency=couplingStrength,qSystem=hs)
    nQubHeisenberg.createTerm(operator=[qg.sigmaz, qg.sigmaz], frequency=couplingStrength,qSystem=hs)

In [ ]:
s1 = qg.freeEvolution(superSys=nQubExchange)

s2 = qg.qProtocol(superSys=nQubExchange)
exchangeCouplings = list(nQubExchange.terms.values())
ind = 0
while ind in range(len(exchangeCouplings)):
    u1 = qg.freeEvolution(superSys=nQubExchange)
    u1.createUpdate(system=[exchangeCouplings[ind], exchangeCouplings[ind+1]], key='frequency', value=couplingStrength/2)
    u1.createUpdate(system=exchangeQubs, key='frequency', value=0)
    s2.addStep(u1)
    ind += 2

xRots = [qg.xGate(system=exchangeQubs, angle=np.pi/2, rotationAxis='x'), 
         qg.xGate(system=exchangeQubs, angle=-np.pi/2, rotationAxis='x')]
yRots = [qg.xGate(system=exchangeQubs, angle=np.pi/2, rotationAxis='y'), 
         qg.xGate(system=exchangeQubs, angle=-np.pi/2, rotationAxis='y')]

protocol = qg.qProtocol(superSys=nQubExchange, steps=[s1,s2,xRots[0],s2,xRots[1],yRots[0],s2,yRots[1]])

In [ ]:
stepSizes = [0.01*(i+1) for i in range(50)]
freqValues = [0.01*(i+1) for i in range(100)]
totalSimTimeV = 5*2*np.pi

sigmaZ = qg.compositeOp(qg.sigmaz(), 2**(numOfQubits-1))
def compute(qsim, args):
    res = qsim.qRes
    res.result = 'state fidelity', qg.fidelityPure(args[0], args[1])
    for key, _ in qsim.subSys.items():
        res.result = [key.name.name+'Exp', qg.expectation(sigmaZ, key.currentState)]

simulation = qg.Simulation()
simulation.initialStateSystem = nQubHeisenberg
simulation.initialState = [0 if x < 1 else 1 for x in range(numOfQubits)]
simulation.delStates = True

simulation.addSubSys(nQubExchange, protocol)
simulation.addSubSys(nQubHeisenberg)

simulation.compute = compute
simulation.totalTime = totalSimTimeV
simulation.stepSize = (stepSizes[0]+0.05)*2*np.pi

# stepSizeSweep = simulation.Sweep.createSweep(system=simulation, sweepKey='stepSize', sweepList=stepSizes)
freqSweep = simulation.Sweep.createSweep(system=[*exchangeQubs, *HeisenbergQubs], sweepKey='frequency', sweepList=freqValues)

In [ ]:
simulation.run(p=True)

In [ ]:
res = simulation.results

In [ ]:
Y, X = np.meshgrid(simulation.timeList, [i for i in range(len(freqSweep.sweepList))])

plt.pcolormesh(X, Y, res['state fidelity'], vmin=0, vmax=1)
plt.colorbar()

plt.xlabel("Frequency $\omega_{c}$")
plt.ylabel("Time")